# Predicting Wine Quality Using Linear and Ordinal Regression

### Authors

| Name | Roll Number |
| - | - |
| Gautam Singh | CS21BTECH11018 |
| Jaswanth Beere | BM21BTECH11007 |

This `.ipynb` file predicts the quality of wine using ordinal regression Packages in Python.

## Package Imports

The required packages for the prediction are specified here

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from statsmodels.miscmodels.ordinal_model import OrderedModel

## Loading Datasets

The `pandas` library is used to load the `csv` files.

In [ ]:
red_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
white_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')

From the dataset, it is clear that the `quality` variable is ordinal with ratings ranging from 3 to 9, while the other variables are real-valued. We require to predict the value of `quality` given the other independent variables. We use an ordinal regression model as well as a linear regression model for the same and compare the performance of both models.

### Preprocessing

Each dataset is split as follows using _proportional sampling_. This is because the ratings are not evenly distributed, and thus using random sampling may not generate an optimal model.
1. 80 percent _training_ data.
2. 20 percent _test_ data.

Before splitting the data, we adjust the ratings so that the smallest rating is zero.

In [ ]:
# Adjust ratings to start from zero onwards
red_df['quality'] -= red_df['quality'].min()
white_df['quality'] -= white_df['quality'].min()
# Proportionally sample from dataset to create training dataset
red_train_df = red_df.groupby('quality').apply(lambda x : x.sample(frac=0.8)).reset_index(drop=True)
white_train_df = white_df.groupby('quality').apply(lambda x : x.sample(frac=0.8)).reset_index(drop=True)
# The remaining data becomes the test dataset
red_test_df = pd.concat([red_df, red_train_df]).drop_duplicates(keep=False).reset_index(drop=True)
white_test_df = pd.concat([white_df, white_train_df]).drop_duplicates(keep=False).reset_index(drop=True)

## Training

### Ordinal Regression

The `statsmodels` library is used to perform ordinal regression on the given dataset.

In [ ]:
# Perform ordinal regression on the training dataset
red_mod_prob = OrderedModel(red_train_df['quality'], red_train_df.loc[:, red_train_df.columns != 'quality'])
# Use the BFGS algorithm to find the maximum likelihood solution
red_res_prob = red_mod_prob.fit(method='bfgs')
# Summarize the results of training
red_res_prob.summary()

In [ ]:
# Perform ordinal regression on the training dataset
white_mod_prob = OrderedModel(white_train_df['quality'], white_train_df.loc[:, white_train_df.columns != 'quality'])
# Use the BFGS algorithm to find the maximum likelihood solution
white_res_prob = white_mod_prob.fit(method='bfgs')
# Summarize the results of training
white_res_prob.summary()

### Linear Regression

We use `scikit-learn` to perform a standard linear regression on the data, by treating `quality` to be a real-valued dependent variable. Notice that 

In [ ]:
# Perform linear regression on training data
red_reg = LinearRegression().fit(red_df.loc[:, red_df.columns != 'quality'], red_df['quality'])
white_reg = LinearRegression().fit(white_df.loc[:, white_df.columns != 'quality'], white_df['quality'])
# Show parameter coefficients for the linear models
# Show constant term and coefficients
print("Parameters for red wine:", red_reg.intercept_, red_reg.coef_)
print("Parameters for white wine:", white_reg.intercept_, white_reg.coef_)

## Testing

### Ordinal Regression

The _root mean squared error_ (RMSE) is a good evaluation metric as per [Gaudette and Japkowicz 2009](https://link.springer.com/chapter/10.1007/978-3-642-01818-3_25), since the ordinal data consists of small integers and deviations from the mean are penalized more severely.

In [ ]:
# Predict class based on maximum probability
red_ord_pred = red_res_prob.predict(red_test_df.loc[:, red_test_df.columns != 'quality']).idxmax(axis=1)
white_ord_pred = white_res_prob.predict(white_test_df.loc[:, white_test_df.columns != 'quality']).idxmax(axis=1)
# Calculate RMSE from correct labels
red_ord_rmse = ((red_ord_pred - red_test_df['quality'])**2).mean()**0.5
white_ord_rmse = ((white_ord_pred - white_test_df['quality'])**2).mean()**0.5
print("RMSE for red wine:", red_ord_rmse)
print("RMSE for white wine:", white_ord_rmse)

### Linear Regression

As per usual, a linear regression model is evaluated using the RMSE.

In [ ]:
# Perform predictions on test data
red_reg_pred = red_reg.predict(red_test_df.loc[:, red_test_df.columns != 'quality'])
white_reg_pred = white_reg.predict(white_test_df.loc[:, white_test_df.columns != 'quality'])
# Compute RMSE of predictions
red_reg_rmse = ((red_reg_pred - red_test_df['quality'])**2).mean()**0.5
white_reg_rmse = ((white_reg_pred - white_test_df['quality'])**2).mean()**0.5
print("RMSE for red wine:", red_reg_rmse)
print("RMSE for white wine:", white_reg_rmse)

## Results

On most runs, it seems that linear regression performs slightly better than ordinal regression, on ordinal data! The possible reasons for this are as follows.

1. The ordinal data corresponds to small integers.
2. The interval classes are spaced apart equally, and linear regression performs equally well in these cases.

However, the downside of using linear regression is a lack of interpretability even though it is a simpler model to work with. Thus, ordinal regression would still be preferred given its high interpretability at the cost of a small increase in RMSE.